In [4]:
import sys, time
sys.path.append('/Users/zhouxinyu/侧边栏文件夹/Code/VSCode/Jupyter/Project/ZF/测试[v2]-22-07-28')
from data_verification import imgs as zf_test_imgs

In [2]:
# 下面两行用不到可以注掉，因为我用的是VS Code，不知道怎么回事找不到模块要自己添加一下
import sys

import cv2
import numpy as np

sys.path.append('/Users/zhouxinyu/Code/Coding/zf_back_py')


class TubeBendingDetect:

    # 裁剪完成后进行分割
    def image_cut(self, img):
        # 从上到下，从左到右，不写就是原始的上下左右边界。带 - 号的是从右边或者从下开始计算距离，-10指切去右边缘长度为10的margin
        img = img[:, :-10, :]
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        lower_blue = np.array([100, 43, 46])
        upper_blue = np.array([124, 255, 255])
        # mask：两个阈值内的变白，其他的变黑，返回一个二值化的图像
        mask = cv2.inRange(hsv, lower_blue, upper_blue)
        #     plt_show(mask)
        img_morph = mask.copy()
        # 腐蚀膨胀，中值滤波
        core = (3, 3)
        cv2.erode(img_morph, core, img_morph, iterations=4)
        #     plt_show(img_morph)
        cv2.dilate(img_morph, core, img_morph, iterations=1)
        #     plt_show(img_morph)
        img_morph = cv2.medianBlur(img_morph, 15)
        #     plt_show(img_morph)
        res = cv2.findContours(img_morph, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        cnts = res[0]  # cnts就是提取到的所有contours组成的list

        # sort the elements in contours by the area of a contour, descending order
        cnts_sort = sorted(cnts, key=cv2.contourArea, reverse=True)
        # 生成一个与img相同shape的全0数组white_img
        white_img = np.zeros_like(img)
        #     plt_show0(white_img)
        white_img[:, :, :] = 0  # 变成一张纯黑的图
        # 第三个参数0代表：绘制cnts_sort里的第一个contour
        # 最后一个参数-1代表：绘制contour时，填充轮廓内部
        cv2.drawContours(white_img, cnts_sort, 0, (255, 255, 255), -1)  # 会在白底上画出黑色的区域
        #     plt_show0(white_img)
        black_img = np.zeros_like(img)
        black_img[:, :, :] = 255  # 变成一张纯白的图
        #     plt_show0(black_img)
        black_img_copy = cv2.cvtColor(black_img, cv2.COLOR_BGR2GRAY)  # 变成了一张纯黑的图
        #     plt_show(black_img_copy)
        cv2.drawContours(black_img_copy, cnts_sort, 0, (0, 0, 0),
                         -1)  # 这里的颜色只要不是(255, 255, 255)，就都是白色；(255, 255, 255)画出黑色
        #     cv2.imwrite('black_img.png', black_img_copy)
        #     plt_show(black_img_copy)
        res = cv2.findContours(black_img_copy, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

        #     cv2.imwrite('black_img.png', black_img_copy)
        img_copy = img.copy()
        cv2.drawContours(img_copy, res[0][0], -1, (0, 255, 0), 10)
        #     cv2.imwrite('img_copy.png', img_copy)
        #     plt_show0(img_copy)
        x, y, w, h = cv2.boundingRect(res[0][0])
        pure_img = img.copy()
        # 获得方框
        cv2.rectangle(pure_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #     cv2.imwrite('pure_img.png', pure_img)
        #     plt_show0(pure_img)
        ## 灰度图
        gray_pure_img = cv2.cvtColor(pure_img, cv2.COLOR_BGR2GRAY)
        #     cv2.imwrite('gray_pure_img.png', gray_pure_img)
        #     plt_show(gray_pure_img)
        # 裁剪
        cuted_image = img[y:y + h, x:x + w]
        #     cv2.imwrite('cuted_image.png', cuted_image)
        #     plt_show0(cuted_image)
        bounding = res[0][0]  # 最大的contour

        # 让contour的所有点的坐标移动，以配合裁剪出来的新图片
        def bounding_move(bounding_point):
            bounding_point[0, 0] -= x  # 裁剪范围的左上角的x坐标
            bounding_point[0, 1] -= y
            return bounding_point

        bounding = np.array(list(map(bounding_move, bounding)))
        return cuted_image, bounding

    def fill_black(self, image, bounding):
        stencil = np.zeros(image.shape).astype(image.dtype)
        color = (255, 255, 255)
        cv2.fillPoly(stencil, [bounding], color)
        result = cv2.bitwise_and(image, stencil)
        return result

    def delete_white(self, rawImage):
        #     plt_show0(rawImage)
        # 高斯去噪
        image = cv2.GaussianBlur(rawImage, (3, 3), 0)
        # plt_show0(image)

        height, width, color = image.shape

        image_hsv = image.copy()
        image_hsv = cv2.cvtColor(image_hsv, cv2.COLOR_RGB2HSV)

        lower_blue = np.array([0, 0, 180])
        upper_blue = np.array([180, 255, 255])
        # mask：两个阈值内的变白，其他的变黑，返回一个二值化的图像
        mask = cv2.inRange(image_hsv, lower_blue, upper_blue)
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # plt_show0(mask)

        stencil = cv2.bitwise_not(mask)
        # plt_show0(stencil)

        image = cv2.bitwise_and(image, stencil)
        # plt_show0(image)

        xPosition = 0
        for x in range(0, width, 5):
            cnt = 0
            for y in range(0, height):
                if image[y][x][0] > 0:
                    cnt += 1
            if cnt > 125:
                xPosition = x
                break
        #     print("xPos",xPosition)
        return xPosition

    def get_two_lines(self, fill_black_right):
        height, width, color = fill_black_right.shape
        fill_black_right_HSV = cv2.cvtColor(fill_black_right, cv2.COLOR_BGR2HSV)
        step = 5 #original step = 5

        # left line ---------------------------------------
        left_line_points_x = []
        left_line_points_y = []
        left_x_start = width//20 # 1/20
        left_x_end = width//5 # 4/20
        for x in range(left_x_start,left_x_end,step):
            left_line_points_x.append(x)
            temp_top = -1
            temp_bottom = -1
            for y in range(0, height):
                if fill_black_right_HSV[y][x][2] != 0:
                    temp_top = y
                    break
            for y in range(height - 1, 0, -1):
                if fill_black_right_HSV[y][x][2] != 0:
                    temp_bottom = y
                    break
            left_line_points_y.append( (temp_top + temp_bottom) // 2 )
        
        #right line ------------------------------------------
        right_line_points_x = []
        right_line_points_y = []
        right_x_start = width - width//5 # 16/20
        right_x_end = width - width//20 # 19/20
        for x in range(right_x_start,right_x_end,step):
            right_line_points_x.append(x)
            temp_top = -1
            temp_bottom = -1
            for y in range(0, height):
                if fill_black_right_HSV[y][x][2] != 0:
                    temp_top = y
                    break
            for y in range(height - 1, 0, -1):
                if fill_black_right_HSV[y][x][2] != 0:
                    temp_bottom = y
                    break
            right_line_points_y.append( (temp_top + temp_bottom) // 2 )
        
        #fitting ------------------------------------------
        left_line = np.polyfit(left_line_points_x, left_line_points_y, 1)
        right_line = np.polyfit(right_line_points_x,right_line_points_y,1)
        
        left_point1_x = 0
        left_point1_y = left_line[1]
        left_point2_x = 1
        left_point2_y = left_line[0] + left_line[1]
        
        right_point1_x = 0
        right_point1_y = right_line[1]
        right_point2_x = 1
        right_point2_y = right_line[0] + right_line[1]
        
        return [[left_point1_x,left_point1_y],[left_point2_x,left_point2_y],[right_point1_x,right_point1_y],[right_point2_x,right_point2_y]]


    def vector_angle(self, line1: list, line2: list):
        """
        计算两条直线的夹角，
        :param line1: 直线一[x1, y1, x2, y2]
        :param line2: 直线二 [x1, y1, x2, y2]
        :return:
        """

        vector1 = np.array([line1[2] - line1[0], line1[3] - line1[1]])  # (x2-x1,y2-y1)
        vector2 = np.array([line2[2] - line2[0], line2[3] - line2[1]])
        return np.arccos(
            vector1 @ vector2 / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

    def process_image(self, image):
        cuted_image, bounding = self.image_cut(image)
        fill_b = self.fill_black(cuted_image, bounding)
        xPosition = self.delete_white(fill_b)
        margin = 50
        height, width, color = fill_b.shape
        fill_black_right = fill_b[0:height, xPosition + margin:width - margin]
        # plt_show0(fill_black_right)
        fourpoints = self.get_two_lines(fill_black_right)
        line1 = [fourpoints[0][0], fourpoints[0][1], fourpoints[1][0], fourpoints[1][1]]
        line2 = [fourpoints[2][0], fourpoints[2][1], fourpoints[3][0], fourpoints[3][1]]
        a = self.vector_angle(line1, line2)
        # print(a)
        pi = 3.1415926
        result = a / pi * 180
        return bool(result < 20), result

tb = TubeBendingDetect()
# cha, du = tb.process_image("/Users/zhouxinyu/论文:电子资料/项目/电缆ZF项目/数据/Image/线芯出观察筒 (3).tif")
# logging.info("cha, du:", cha, du)


In [5]:
for i, filename in enumerate(zf_test_imgs.line_pollution):
    t1 = time.time()
    res = tb.process_image(cv2.imread(filename))
    t2 = time.time()
    print(f'{i}: {t2 - t1}, {res}')

0: 0.3143160343170166, (True, 0.680112770013112)
1: 2.893364906311035, (True, 0.47779519316748637)
2: 0.4748408794403076, (True, 0.5087187716239998)
3: 0.452603816986084, (True, 0.9643124465453802)
4: 0.48346590995788574, (True, 1.1809670312376337)
5: 0.41590213775634766, (True, 0.9249186988654521)
6: 0.4082920551300049, (True, 0.7460575407333546)
7: 0.330974817276001, (True, 0.8885890913493621)
8: 0.39576077461242676, (True, 0.7268321211176014)
9: 0.31380796432495117, (True, 1.208887290779275)
10: 0.3271632194519043, (True, 1.4454720951017468)
11: 0.43251705169677734, (True, 0.37502855487542597)
12: 0.47960424423217773, (True, 0.055365345724022436)
13: 0.480299711227417, (True, 1.7421580918743504)
14: 0.3414731025695801, (True, 0.35451666092640965)
15: 0.39038610458374023, (True, 0.7433657754488102)
16: 0.4783189296722412, (True, 1.244882282115967)
17: 0.34645605087280273, (True, 1.323243612007242)
18: 0.35526394844055176, (True, 0.5653737034085122)
19: 0.4154670238494873, (True, 0.46

In [24]:
import time

t1 = time.time()
time.sleep(4)
t2 = time.time()
period = t2 - t1
print(period)

4.001936197280884
